Our IMDB network has been trained

Our IMDB network has been trained and is ready for deployment. Perhaps get a machine to rate chucks of text as positive or negative. 

In [1]:
from tensorflow.keras.datasets import imdb

((train_data, train_labels), (test_data, test_labels)) = imdb.load_data(num_words = 10000)

import numpy as np

def vectorize_sequences(sequences, dimension = 10000):
    results = np.zeros( (len(sequences), dimension) )
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 4, batch_size = 512)

Epoch 1/4
49/49 [==============================] - 1s 11ms/step - loss: 0.4552 - accuracy: 0.8173
Epoch 2/4
49/49 [==============================] - 1s 11ms/step - loss: 0.2551 - accuracy: 0.9113
Epoch 3/4
49/49 [==============================] - 1s 11ms/step - loss: 0.1969 - accuracy: 0.9292
Epoch 4/4
49/49 [==============================] - 1s 11ms/step - loss: 0.1657 - accuracy: 0.9392


In [4]:
model.predict(x_test[:10])

array([[0.16038436],
       [0.9999798 ],
       [0.915747  ],
       [0.83915466],
       [0.9537009 ],
       [0.8669092 ],
       [0.99947035],
       [0.01265699],
       [0.9718611 ],
       [0.9978831 ]], dtype=float32)

`predict` gives network output. Here we see the first ten predictions as values between 0 and 1.

In [5]:
test_labels[0:10]

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

Here are the targets - the first ten labels.

In [6]:
print('review', '\t', 'prediction', '\t', '\t', 'label')
for i in range(20):
    y_pred = model.predict(x_test[i:i+1])[0][0]

    print(i, '\t', y_pred,'\t', round(y_pred), '\t', test_labels[i])

review 	 prediction 	 	 label
0 	 0.16038433 	 0.0 	 0
1 	 0.9999798 	 1.0 	 1
2 	 0.91574717 	 1.0 	 1
3 	 0.83915484 	 1.0 	 0
4 	 0.9537009 	 1.0 	 1
5 	 0.86690915 	 1.0 	 1
6 	 0.9994703 	 1.0 	 1
7 	 0.012656927 	 0.0 	 0
8 	 0.9718611 	 1.0 	 0
9 	 0.9978831 	 1.0 	 1
10 	 0.9553802 	 1.0 	 1
11 	 0.007366717 	 0.0 	 0
12 	 0.00028815866 	 0.0 	 0
13 	 0.027026087 	 0.0 	 0
14 	 0.9992556 	 1.0 	 1
15 	 0.00021713972 	 0.0 	 0
16 	 0.9063751 	 1.0 	 1
17 	 0.7856398 	 1.0 	 0
18 	 0.0067875385 	 0.0 	 0
19 	 0.05469632 	 0.0 	 0


Let's look at some examples... the third column is the rounded network prediction. Quite a lot of hits, but a few mistakes too.

In [2]:
def find_first(prediction, label):
    for i in range(len(x_test)):
        y_pred = round(model.predict(x_test[i:i+1])[0][0])
        if(y_pred == prediction and test_labels[i] == label):
            return i

In [3]:
def print_review(n):
    
    word_index = imdb.get_word_index()
    reverse_word_index = dict([(value, key) for (key,value) in word_index.items()])
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in test_data[n]])
    
    y_pred = model.predict(x_test[n:n+1])[0][0]
    print('predicted: ', np.round(y_pred), '\tlabel:', test_labels[n])
    print(decoded_review, '\n')   

In [7]:
print_review(find_first(prediction=0, label=0))

predicted:  0.0 	label: 0
? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss 



The first successful prediction of a negative review. 

In [8]:
print_review(find_first(prediction=1, label=1))

predicted:  1.0 	label: 1
? this film requires a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances ? the sandy dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere ? with sexual tension and psychological ? it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary michael burns does a good job as the mute young man regular altman player michael murphy has a small part the ? moody set fits the content of the story very well in short this movie is a powerful study of loneliness sexual ? and desperation be patient ? up the a

The first successful positive prediction.

In [9]:
print_review(find_first(prediction=0, label=1))

predicted:  0.0 	label: 1
? how managed to avoid attention remains a mystery a potent mix of comedy and crime this one takes chances where tarantino plays it safe with the hollywood formula the risks don't always pay off one character in one sequence comes off ? silly and falls flat in the lead role thomas jane gives a wonderful and complex performance and two brief appearances by mickey rourke hint at the high potential of this much under and ? used actor here's a director one should keep one's eye on 



The network predicted negative but the actual label is positive. What do you think?

In [10]:
print_review(find_first(prediction=1, label=0))

predicted:  1.0 	label: 0
? i generally love this type of movie however this time i found myself wanting to kick the screen since i can't do that i will just complain about it this was absolutely idiotic the things that happen with the dead kids are very cool but the alive people are absolute idiots i am a grown man pretty big and i can defend myself well however i would not do half the stuff the little girl does in this movie also the mother in this movie is reckless with her children to the point of neglect i wish i wasn't so angry about her and her actions because i would have otherwise enjoyed the flick what a number she was take my advise and fast forward through everything you see her do until the end also is anyone else getting sick of watching movies that are filmed so dark anymore one can hardly see what is being filmed as an audience we are ? involved with the actions on the screen so then why the hell can't we have night vision 



And the network wrongly classed this review as positive.